# Model 1: Trump Personality Profile & Compatibility Predictor

## Overview
This notebook builds a **predictive model** that:
1. Extracts Trump's **Big Five personality traits** from his speech patterns
2. Creates a **personality compatibility matrix** based on psychological research
3. **Predicts** how Trump would respond to different personality types

## Scientific Basis
- **LIWC (Linguistic Inquiry and Word Count)** patterns → personality inference
- **Big Five Personality Model** (OCEAN): Openness, Conscientiousness, Extraversion, Agreeableness, Neuroticism
- **Interpersonal Circumplex Theory** for compatibility predictions


In [ ]:
import pandas as pd
import numpy as np
import json
from pathlib import Path
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

print("Libraries loaded successfully!")


## Step 1: Load Speech Features Data


In [ ]:
# Find the most recent features file
data_dir = Path('../data/transformed')
feature_files = list(data_dir.glob('speeches_features_complete_*.json'))
latest_file = max(feature_files, key=lambda x: x.stat().st_mtime)

print(f"Loading: {latest_file.name}")

with open(latest_file, 'r', encoding='utf-8') as f:
    speeches_data = json.load(f)

df = pd.DataFrame(speeches_data)
print(f"Loaded {len(df)} speeches with {len(df.columns)} features")


## Step 2: Extract Trump's Big Five Personality Profile

### Linguistic Markers for Big Five (Based on LIWC Research)

| Trait | High Score Indicators | Low Score Indicators |
|-------|----------------------|---------------------|
| **Openness** | Complex sentences, diverse vocabulary, abstract words | Simple language, concrete terms |
| **Conscientiousness** | Achievement words, future tense, organized speech | Casual language, hedging |
| **Extraversion** | "I" pronouns, positive emotions, social references | Tentative language, fewer words |
| **Agreeableness** | "We" pronouns, affiliation words, positive affect | Anger words, negations, "I" focus |
| **Neuroticism** | Negative emotions, anxiety words, hedging | Certainty markers, positive affect |


In [ ]:
def compute_big_five_from_speech(df):
    """
    Compute Big Five personality traits from linguistic features.
    Based on LIWC research correlations with personality.
    
    Returns scores from 0-100 for each trait.
    """
    
    # Aggregate features across all speeches
    features = {
        # Pronoun patterns
        'i_we_ratio': df['pronoun_i_we_ratio'].mean(),
        'first_singular': df['pronoun_first_singular'].mean(),
        'first_plural': df['pronoun_first_plural'].mean(),
        'total_pronouns': df['pronoun_total'].mean(),
        
        # Linguistic complexity
        'lexical_diversity': df['lexical_diversity'].mean(),
        'avg_sentence_length': df['avg_sentence_length'].mean(),
        'readability': df['readability_flesch_reading_ease'].mean(),
        
        # Certainty and power
        'certainty_markers': df['certainty_markers'].mean(),
        'power_words': df['power_words'].mean(),
        'affiliation_words': df['affiliation_words'].mean(),
        'power_affiliation_ratio': df['power_affiliation_ratio'].mean(),
        
        # Modal verbs (certainty indicators)
        'modal_will': df['modal_will'].mean(),
        'modal_must': df['modal_must'].mean(),
        'modal_should': df['modal_should'].mean(),
        'modal_can': df['modal_can'].mean(),
        
        # Sentiment
        'sentiment_pos': df['sentiment_pos'].mean(),
        'sentiment_neg': df['sentiment_neg'].mean(),
        'sentiment_variance': df['sentiment_variance'].mean(),
        
        # Rhetorical patterns
        'superlatives': df['superlative_count'].mean(),
        'exclamations': df['exclamation_count'].mean(),
        'questions': df['question_count'].mean(),
        'all_caps': df['all_caps_words'].mean(),
        'repetition': df['repetition_density'].mean(),
    }
    
    # ========== COMPUTE BIG FIVE SCORES ==========
    
    # 1. OPENNESS (Intellectual curiosity, creativity)
    # High: lexical diversity, complex sentences, varied vocabulary
    # Trump: LOW - simple language, repetitive, concrete
    openness_raw = (
        features['lexical_diversity'] * 100 +  # diversity → openness
        (100 - features['readability']) * 0.5 +  # harder to read → more open
        features['avg_sentence_length'] * 2  # longer sentences → more open
    ) / 3
    openness = min(100, max(0, openness_raw * 2))  # Scale to 0-100
    
    # 2. CONSCIENTIOUSNESS (Organization, achievement-focus)
    # High: future tense (will), achievement words, organized
    # Trump: MODERATE-HIGH - talks about winning, future plans
    conscientiousness_raw = (
        features['modal_will'] * 3 +  # future planning
        features['certainty_markers'] * 5 +  # decisiveness
        features['superlatives'] * 2  # achievement language
    ) / 10
    conscientiousness = min(100, max(0, conscientiousness_raw * 4))
    
    # 3. EXTRAVERSION (Assertiveness, sociability, positive emotions)
    # High: "I" pronouns, positive affect, social words, exclamations
    # Trump: VERY HIGH - dominant, assertive, expressive
    extraversion_raw = (
        features['first_singular'] / 10 +  # "I" focus → assertiveness
        features['sentiment_pos'] * 100 +  # positive emotions
        features['exclamations'] * 5 +  # expressiveness
        features['all_caps'] * 2 +  # emphasis/loudness
        features['superlatives'] * 3  # dramatic expression
    ) / 5
    extraversion = min(100, max(0, extraversion_raw * 2))
    
    # 4. AGREEABLENESS (Cooperation, trust, warmth)
    # High: "we" pronouns, affiliation, low anger
    # Trump: LOW - competitive, combative, self-focused
    agreeableness_raw = (
        (1 / (features['i_we_ratio'] + 0.1)) * 10 +  # more "we" → agreeable
        features['affiliation_words'] * 3 +  # teamwork language
        (1 - features['sentiment_neg']) * 50 +  # less negativity → agreeable
        (100 - features['power_affiliation_ratio'] * 100) * 0.5  # less power-focused
    ) / 4
    agreeableness = min(100, max(0, agreeableness_raw))
    
    # 5. NEUROTICISM (Emotional instability, anxiety)
    # High: negative emotions, anxiety words, hedging
    # Trump: MODERATE - emotional volatility, but projects confidence
    neuroticism_raw = (
        features['sentiment_neg'] * 200 +  # negative emotions
        features['sentiment_variance'] * 100 +  # emotional swings
        features['all_caps'] * 1 +  # emotional intensity
        (100 - features['certainty_markers'] * 5)  # uncertainty → neuroticism
    ) / 4
    neuroticism = min(100, max(0, neuroticism_raw))
    
    return {
        'openness': round(openness, 1),
        'conscientiousness': round(conscientiousness, 1),
        'extraversion': round(extraversion, 1),
        'agreeableness': round(agreeableness, 1),
        'neuroticism': round(neuroticism, 1),
        'raw_features': features
    }

# Compute Trump's personality profile
trump_profile = compute_big_five_from_speech(df)

print("="*60)
print("TRUMP'S BIG FIVE PERSONALITY PROFILE")
print("="*60)
print(f"\n{'Trait':<20} {'Score':>10} {'Level':>15}")
print("-"*45)

def get_level(score):
    if score >= 75: return "VERY HIGH"
    if score >= 55: return "HIGH"
    if score >= 45: return "MODERATE"
    if score >= 25: return "LOW"
    return "VERY LOW"

for trait in ['openness', 'conscientiousness', 'extraversion', 'agreeableness', 'neuroticism']:
    score = trump_profile[trait]
    level = get_level(score)
    print(f"{trait.capitalize():<20} {score:>10.1f} {level:>15}")


In [ ]:
# Visualize Trump's Big Five Profile
fig, ax = plt.subplots(figsize=(10, 6))

traits = ['Openness', 'Conscientiousness', 'Extraversion', 'Agreeableness', 'Neuroticism']
scores = [trump_profile['openness'], trump_profile['conscientiousness'], 
          trump_profile['extraversion'], trump_profile['agreeableness'], 
          trump_profile['neuroticism']]

colors = ['#FF6B6B', '#4ECDC4', '#FFE66D', '#95E1D3', '#B19CD9']

bars = ax.barh(traits, scores, color=colors, edgecolor='black', linewidth=1.5)

# Add score labels
for bar, score in zip(bars, scores):
    ax.text(score + 2, bar.get_y() + bar.get_height()/2, f'{score:.0f}', 
            va='center', fontsize=12, fontweight='bold')

ax.set_xlim(0, 110)
ax.set_xlabel('Score (0-100)', fontsize=12)
ax.set_title("Trump's Big Five Personality Profile\n(Extracted from 43 Speech Transcripts)", 
             fontsize=14, fontweight='bold')

# Add reference lines
ax.axvline(x=50, color='gray', linestyle='--', alpha=0.5, label='Average')

plt.tight_layout()
plt.savefig('../data/results/trump_big_five_profile.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nProfile saved to: data/results/trump_big_five_profile.png")


## Step 3: Build Personality Compatibility Predictor

### Psychological Research on Compatibility
Based on interpersonal psychology research:

- **Similar Extraversion levels** → better rapport
- **Complementary Dominance** → one dominant, one submissive works better than two dominant
- **High Agreeableness** person can work with anyone
- **Low Agreeableness + Low Agreeableness** → conflict


In [ ]:
class PersonalityCompatibilityPredictor:
    """
    Predicts how Trump will respond to different personality types.
    
    Based on:
    1. Interpersonal Circumplex Theory (Dominance vs Affiliation)
    2. Big Five compatibility research
    3. Trump-specific patterns from speech analysis
    """
    
    def __init__(self, trump_profile):
        self.trump = trump_profile
        
        # Trump's key characteristics derived from Big Five
        self.trump_dominance = (self.trump['extraversion'] + (100 - self.trump['agreeableness'])) / 2
        self.trump_warmth = (self.trump['agreeableness'] + self.trump['extraversion']) / 2
        
        print(f"Trump's Interpersonal Style:")
        print(f"  Dominance: {self.trump_dominance:.1f}/100 (HIGH - assertive, competitive)")
        print(f"  Warmth: {self.trump_warmth:.1f}/100 (MODERATE - can be charming but combative)")
    
    def compute_compatibility(self, other_profile):
        """
        Compute compatibility score between Trump and another personality.
        """
        
        # Other person's interpersonal style
        other_dominance = (other_profile['extraversion'] + (100 - other_profile['agreeableness'])) / 2
        other_warmth = (other_profile['agreeableness'] + other_profile['extraversion']) / 2
        
        # ===== COMPATIBILITY FACTORS =====
        
        # Factor 1: Dominance Complementarity (0-25 points)
        dominance_diff = self.trump_dominance - other_dominance
        if dominance_diff > 30:
            dominance_score = 25
            dominance_note = "Deferential - Trump will appreciate respect"
        elif dominance_diff > 10:
            dominance_score = 20
            dominance_note = "Respectful but not pushover"
        elif dominance_diff > -10:
            dominance_score = 10
            dominance_note = "Power struggle likely"
        else:
            dominance_score = 5
            dominance_note = "Will trigger competitive response"
        
        # Factor 2: Agreeableness Match (0-25 points)
        if other_profile['agreeableness'] > 70:
            agree_score = 25
            agree_note = "Accommodating - will avoid conflict"
        elif other_profile['agreeableness'] > 50:
            agree_score = 18
            agree_note = "Diplomatic - can manage tensions"
        elif other_profile['agreeableness'] > 30:
            agree_score = 10
            agree_note = "Both competitive - friction expected"
        else:
            agree_score = 5
            agree_note = "Two low-agreeableness personalities - conflict inevitable"
        
        # Factor 3: Conscientiousness Alignment (0-20 points)
        if other_profile['conscientiousness'] > 60:
            consc_score = 20
            consc_note = "Competent/Organized - earns respect"
        elif other_profile['conscientiousness'] > 40:
            consc_score = 15
            consc_note = "Adequate competence"
        else:
            consc_score = 8
            consc_note = "May be seen as incompetent"
        
        # Factor 4: Emotional Stability (0-15 points)
        if other_profile['neuroticism'] < 30:
            neuro_score = 15
            neuro_note = "Emotionally stable - won't be rattled"
        elif other_profile['neuroticism'] < 50:
            neuro_score = 12
            neuro_note = "Moderate stability"
        else:
            neuro_score = 7
            neuro_note = "Anxious - may be pushed around"
        
        # Factor 5: Status/Achievement Perception (0-15 points)
        status_proxy = (other_profile['extraversion'] + other_profile['conscientiousness']) / 2
        if status_proxy > 60:
            status_score = 15
            status_note = "Projects success - earns initial respect"
        elif status_proxy > 40:
            status_score = 10
            status_note = "Average status perception"
        else:
            status_score = 5
            status_note = "May be dismissed as 'low energy'"
        
        total_score = dominance_score + agree_score + consc_score + neuro_score + status_score
        
        return {
            'compatibility_score': total_score,
            'factors': {
                'dominance_complementarity': {'score': dominance_score, 'note': dominance_note},
                'agreeableness_match': {'score': agree_score, 'note': agree_note},
                'conscientiousness_respect': {'score': consc_score, 'note': consc_note},
                'emotional_stability': {'score': neuro_score, 'note': neuro_note},
                'status_perception': {'score': status_score, 'note': status_note}
            },
            'other_interpersonal': {
                'dominance': other_dominance,
                'warmth': other_warmth
            }
        }
    
    def predict_response(self, other_profile):
        """
        Predict Trump's behavioral response to a person with given personality.
        """
        
        compat = self.compute_compatibility(other_profile)
        score = compat['compatibility_score']
        other_dominance = compat['other_interpersonal']['dominance']
        
        # Determine response type
        if score >= 75:
            response_type = "COOPERATIVE"
            description = "Trump will likely be friendly and receptive"
            confidence = min(95, 60 + score * 0.4)
        elif score >= 55:
            response_type = "TRANSACTIONAL"
            description = "Trump will engage if there's something in it for him"
            confidence = min(85, 50 + score * 0.35)
        elif score >= 40:
            if other_dominance > self.trump_dominance - 10:
                response_type = "COMPETITIVE"
                description = "Trump will try to assert dominance and 'win'"
            else:
                response_type = "DISMISSIVE"
                description = "Trump may ignore or belittle"
            confidence = min(75, 40 + score * 0.3)
        else:
            if other_dominance > self.trump_dominance:
                response_type = "HOSTILE"
                description = "Trump will likely attack and undermine"
            else:
                response_type = "CONTEMPTUOUS"
                description = "Trump will show open disdain"
            confidence = min(80, 30 + (100 - score) * 0.5)
        
        recommendations = self._generate_recommendations(compat, response_type)
        
        return {
            'compatibility_score': score,
            'response_type': response_type,
            'description': description,
            'confidence': round(confidence, 1),
            'factor_breakdown': compat['factors'],
            'recommendations': recommendations
        }
    
    def _generate_recommendations(self, compat, response_type):
        """Generate strategic recommendations for interacting with Trump"""
        
        recs = []
        factors = compat['factors']
        
        if factors['dominance_complementarity']['score'] < 15:
            recs.append("AVOID direct power challenges - let him feel in control")
            recs.append("Use indirect influence rather than confrontation")
        
        if factors['agreeableness_match']['score'] < 15:
            recs.append("Show respect even while disagreeing")
            recs.append("Pick battles carefully - concede on small things")
        
        if factors['status_perception']['score'] < 12:
            recs.append("Establish credibility through achievements early")
            recs.append("Reference past successes to gain respect")
        
        recs.append("Frame proposals as 'wins' for him personally")
        recs.append("Use flattery strategically but authentically")
        
        if response_type in ['HOSTILE', 'COMPETITIVE']:
            recs.append("Consider having a mutual ally make the introduction")
            recs.append("Don't show weakness - maintain composure under pressure")
        
        return recs[:5]

# Initialize predictor
predictor = PersonalityCompatibilityPredictor(trump_profile)


## Step 4: Test Predictions with Different Personality Types


In [ ]:
# Define test personality archetypes
test_profiles = {
    "Submissive Agreeable (Yes-Man)": {
        'openness': 40,
        'conscientiousness': 50,
        'extraversion': 30,
        'agreeableness': 85,
        'neuroticism': 50
    },
    "Dominant Competitor (Alpha)": {
        'openness': 50,
        'conscientiousness': 70,
        'extraversion': 90,
        'agreeableness': 20,
        'neuroticism': 30
    },
    "Diplomatic Professional": {
        'openness': 65,
        'conscientiousness': 80,
        'extraversion': 55,
        'agreeableness': 70,
        'neuroticism': 25
    },
    "Intellectual Critic": {
        'openness': 90,
        'conscientiousness': 75,
        'extraversion': 45,
        'agreeableness': 35,
        'neuroticism': 40
    },
    "Anxious Subordinate": {
        'openness': 50,
        'conscientiousness': 60,
        'extraversion': 25,
        'agreeableness': 60,
        'neuroticism': 80
    },
    "Charismatic Leader": {
        'openness': 70,
        'conscientiousness': 65,
        'extraversion': 85,
        'agreeableness': 55,
        'neuroticism': 20
    }
}

print("="*80)
print("TRUMP RESPONSE PREDICTIONS FOR DIFFERENT PERSONALITY TYPES")
print("="*80)

results = []
for name, profile in test_profiles.items():
    prediction = predictor.predict_response(profile)
    results.append({
        'archetype': name,
        'compatibility': prediction['compatibility_score'],
        'response': prediction['response_type'],
        'confidence': prediction['confidence']
    })
    
    print(f"\n{'─'*80}")
    print(f"Personality: {name}")
    print(f"{'─'*80}")
    print(f"  Compatibility Score: {prediction['compatibility_score']}/100")
    print(f"  Predicted Response: {prediction['response_type']}")
    print(f"  Description: {prediction['description']}")
    print(f"  Confidence: {prediction['confidence']}%")
    print(f"\n  Factor Breakdown:")
    for factor, data in prediction['factor_breakdown'].items():
        print(f"    • {factor}: {data['score']} pts - {data['note']}")
    print(f"\n  Recommendations:")
    for i, rec in enumerate(prediction['recommendations'], 1):
        print(f"    {i}. {rec}")


In [ ]:
# Visualize compatibility scores
fig, ax = plt.subplots(figsize=(12, 6))

results_df = pd.DataFrame(results)
colors_map = {
    'COOPERATIVE': '#2ECC71',
    'TRANSACTIONAL': '#F39C12',
    'COMPETITIVE': '#E74C3C',
    'DISMISSIVE': '#9B59B6',
    'HOSTILE': '#C0392B',
    'CONTEMPTUOUS': '#7F8C8D'
}

bar_colors = [colors_map.get(r, '#3498DB') for r in results_df['response']]
bars = ax.barh(results_df['archetype'], results_df['compatibility'], color=bar_colors, edgecolor='black')

# Add score labels
for bar, score, response in zip(bars, results_df['compatibility'], results_df['response']):
    ax.text(score + 1, bar.get_y() + bar.get_height()/2, 
            f'{score} ({response})', va='center', fontsize=10)

ax.set_xlim(0, 110)
ax.set_xlabel('Compatibility Score', fontsize=12)
ax.set_title("Trump's Predicted Response by Personality Type", fontsize=14, fontweight='bold')

# Add threshold lines
ax.axvline(x=75, color='green', linestyle='--', alpha=0.5, label='Cooperative threshold')
ax.axvline(x=55, color='orange', linestyle='--', alpha=0.5, label='Transactional threshold')
ax.axvline(x=40, color='red', linestyle='--', alpha=0.5, label='Conflict threshold')
ax.legend(loc='lower right')

plt.tight_layout()
plt.savefig('../data/results/trump_compatibility_predictions.png', dpi=150, bbox_inches='tight')
plt.show()


## Step 5: Interactive Predictor Function (for GUI)


In [ ]:
def predict_trump_compatibility(openness, conscientiousness, extraversion, agreeableness, neuroticism):
    """
    Main prediction function for the GUI.
    
    Parameters (all 0-100):
    - openness: Intellectual curiosity, creativity
    - conscientiousness: Organization, dependability, achievement
    - extraversion: Assertiveness, sociability, energy
    - agreeableness: Cooperation, trust, empathy
    - neuroticism: Emotional instability, anxiety
    
    Returns: Complete prediction with recommendations
    """
    
    profile = {
        'openness': openness,
        'conscientiousness': conscientiousness,
        'extraversion': extraversion,
        'agreeableness': agreeableness,
        'neuroticism': neuroticism
    }
    
    return predictor.predict_response(profile)

# Test the function
print("\n" + "="*60)
print("INTERACTIVE PREDICTOR TEST")
print("="*60)

# Example: A strong, confident negotiator
result = predict_trump_compatibility(
    openness=60,
    conscientiousness=75,
    extraversion=70,
    agreeableness=50,
    neuroticism=20
)

print(f"\nInput Profile: Confident Negotiator")
print(f"  O=60, C=75, E=70, A=50, N=20")
print(f"\nPrediction:")
print(f"  Response Type: {result['response_type']}")
print(f"  Compatibility: {result['compatibility_score']}/100")
print(f"  Confidence: {result['confidence']}%")
print(f"  {result['description']}")


## Step 6: Save Model Artifacts


In [ ]:
from datetime import datetime

# Save Trump's personality profile
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

profile_output = {
    'trump_big_five': {
        'openness': trump_profile['openness'],
        'conscientiousness': trump_profile['conscientiousness'],
        'extraversion': trump_profile['extraversion'],
        'agreeableness': trump_profile['agreeableness'],
        'neuroticism': trump_profile['neuroticism']
    },
    'trump_interpersonal': {
        'dominance': predictor.trump_dominance,
        'warmth': predictor.trump_warmth
    },
    'derived_from': {
        'num_speeches': len(df),
        'raw_features': trump_profile['raw_features']
    },
    'methodology': 'LIWC-based Big Five inference from speech transcripts',
    'created_at': timestamp
}

output_path = Path(f'../data/results/trump_personality_profile_{timestamp}.json')
with open(output_path, 'w') as f:
    json.dump(profile_output, f, indent=2, default=str)

print(f"✓ Trump's personality profile saved to: {output_path}")

# Save test predictions
predictions_output = {
    'model': 'Personality Compatibility Predictor',
    'test_predictions': []
}

for name, profile in test_profiles.items():
    pred = predictor.predict_response(profile)
    predictions_output['test_predictions'].append({
        'archetype': name,
        'input_profile': profile,
        'prediction': pred
    })

pred_path = Path(f'../data/results/compatibility_predictions_{timestamp}.json')
with open(pred_path, 'w') as f:
    json.dump(predictions_output, f, indent=2, default=str)

print(f"✓ Compatibility predictions saved to: {pred_path}")


## Model Summary

### What This Model Does:
1. **Extracts Trump's Big Five personality** from 43 speech transcripts using LIWC-style linguistic analysis
2. **Maps linguistic features to personality traits** (pronoun usage → extraversion, certainty markers → conscientiousness, etc.)
3. **Computes compatibility scores** based on interpersonal psychology research
4. **Predicts behavioral response** (Cooperative, Transactional, Competitive, Hostile, etc.)
5. **Generates strategic recommendations** for interacting with Trump

### Key Findings:
- Trump scores **HIGH** on Extraversion (assertive, dominant)
- Trump scores **LOW** on Agreeableness (competitive, combative)
- Trump scores **LOW** on Openness (simple, concrete language)
- Best compatibility: **High Agreeableness + Lower Dominance** people
- Worst compatibility: **High Dominance + Low Agreeableness** (Alpha types)

### Prediction Categories:
| Response Type | Compatibility Score | Description |
|---------------|---------------------|-------------|
| COOPERATIVE | 75+ | Friendly and receptive |
| TRANSACTIONAL | 55-74 | Engages if beneficial |
| COMPETITIVE | 40-54 (high dominance) | Power struggle |
| DISMISSIVE | 40-54 (low dominance) | Ignores/belittles |
| HOSTILE | <40 (high dominance) | Attacks/undermines |
| CONTEMPTUOUS | <40 (low dominance) | Open disdain |
